In [12]:
import os
import uuid
import stat
import subprocess
import numpy as np
import pandas as pd
from datetime import datetime

### Make metadata files for the datasets not from GEO

In [13]:
data_table = pd.read_csv('../haase_lab_microarray_datasets.csv', index_col=0)
none_geo_datasets = data_table[data_table['GEO Accession'] == 'No GEO']
# none_geo_datasets

In [14]:
def make_sample_list(exp_dir_name):
                     
    rep = exp_dir_name.split('_')[-1]
    exp_name = '_'.join(exp_dir_name.split('_')[:-1])
    sample_list = []
    for file in os.listdir(os.path.join('../data', exp_dir_name, 'CEL_RAW')):
        sample_name = file.split('_')[3:-1]
        if len(sample_name) > 1:
            sample_name = '_'.join(sample_name)
        else:
            sample_name = sample_name[0]
        for string in ['t', '_', 'min']:
            if string in sample_name:
                if string == 'min':
                    tp = sample_name
                    if len(tp) == 5:
                        tp = '0' + tp
                    elif len(tp) == 4:
                        tp = '00' + tp
                else:
                    tp = sample_name.split(string)[-1]
                    if len(tp) == 2:
                        tp = '0' + tp
                    elif len(tp) == 1:
                        tp = '00' + tp
                    tp += 'min'
        sample_list.append(['_'.join([exp_name, tp, rep]), file])
        
    sorted_sample_list = sorted(sample_list, key=lambda sample_file: sample_file[0])
    return sorted_sample_list


def make_metadata_file(dir_name, output_dir):

    sample_file_lists = make_sample_list(dir_name)
    meta_cols = ['title','geo_accession','file','experiment','condition','replicate','time','organism','characteristics','treatment','growth','sample_type']
    metadata_df = pd.DataFrame(columns=meta_cols)
    
    metadata_df.loc[:,'title'] = [sample_file_combo[0] for sample_file_combo in sample_file_lists]
    metadata_df.loc[:,'geo_accession'] = ['noGEO{}'.format(uuid.uuid4().hex) for n in np.arange(1,len(sample_file_lists)+1)]
    metadata_df.loc[:,'file'] = [sample_file_combo[1] for sample_file_combo in sample_file_lists]
    metadata_df.loc[:,'experiment'] = dir_name
    metadata_df.loc[:,'condition'] = none_geo_datasets.at[dir_name, 'condition']
    metadata_df.loc[:,'replicate'] = none_geo_datasets.at[dir_name, 'replicate']
    metadata_df.loc[:,'organism'] = 'Saccharomyces cerevisiae'
    metadata_df.loc[:,'characteristics'] = none_geo_datasets.at[dir_name, 'characteristics']
    metadata_df.loc[:,'treatment'] = none_geo_datasets.at[dir_name, 'treatment']
    metadata_df.loc[:,'growth'] = none_geo_datasets.at[dir_name, 'growth']
    metadata_df.loc[:,'sample_type'] = none_geo_datasets.at[dir_name, 'sample_type']
    
    time_points = []
    for sample_name in  [sample_file_combo[0] for sample_file_combo in sample_file_lists]:
        time_points.append(int(sample_name.split('_')[-2][:3]))
    
    metadata_df.loc[:,'time'] = time_points
    
    output_file = os.path.join(output_dir, '{}_metadata.tsv'.format(dir_name))
    
    with open(output_file, 'w') as outfile:
        outfile.write('# Metadata for experiment {}\n'.format(dir_name))
        outfile.write('# Generated: {}\n'.format(datetime.today().strftime('%Y-%m-%d %H:%M:%S')))
        outfile.write('# Author: Robert C. Moseley\n#\n#\n')
        metadata_df.to_csv(outfile, sep='\t', index=False)

In [15]:
for dir_name in set(none_geo_datasets.index.tolist()):
    outdir = os.path.join('../data', dir_name)
    make_metadata_file(dir_name, outdir)

### Make metadata files for the datasets taken from GEO

In [16]:
geo_datasets = data_table[data_table['GEO Accession'] != 'No GEO']
geo_datasets
geo_datasets_list = geo_datasets.reset_index()[['Published Paper', 'GEO Accession']].values.tolist()
geo_datasets_list = [list(x) for x in set(tuple(x) for x in geo_datasets_list)]
geo_datasets_list

[['Cho 2019', 'GSE75694'],
 ['Orlando 2008', 'GSE8799'],
 ['Simmons Kovacs 2012', 'GSE32974'],
 ['Cho 2017', 'GSE96997'],
 ['Bristow 2014', 'GSE49650']]

In [17]:
datetime_stamp = datetime.today().strftime('%Y%m%d%H%M%S')
with open(f'batch_metadata_file_{datetime_stamp}.sh', 'w+') as batch_file:
    batch_file.write('#!/bin/sh\n\n')
    batch_file.write('cd transcriptomics_process/get_metadata_from_geo\n\n')
    print('Commands added to bash file:')
    for ds in geo_datasets_list:
        data = ds[0].replace(' ','').lower()
        data_path = os.path.join('../../../data/{}/{}_series_matrix.txt'.format(ds[1], ds[1]))
        if data == 'orlando2008':
            skiprows = 26
        else:
            skiprows = 25

        batch_file.write('python extract_geo_metadata.py -data_path {} -dataset_name {} -skiprows {}\n\n'.format(data_path, data, skiprows))
        print('python extract_geo_metadata.py -data_path {} -dataset_name {} -skiprows {}\n'.format(data_path, data, skiprows))
    batch_file.write('cd -\n')

# Make bash file executable
st = os.stat(f'batch_metadata_file_{datetime_stamp}.sh')
os.chmod(f'batch_metadata_file_{datetime_stamp}.sh', st.st_mode | stat.S_IEXEC)

Commands added to bash file:
python extract_geo_metadata.py -data_path ../../../data/GSE75694/GSE75694_series_matrix.txt -dataset_name cho2019 -skiprows 25

python extract_geo_metadata.py -data_path ../../../data/GSE8799/GSE8799_series_matrix.txt -dataset_name orlando2008 -skiprows 26

python extract_geo_metadata.py -data_path ../../../data/GSE32974/GSE32974_series_matrix.txt -dataset_name simmonskovacs2012 -skiprows 25

python extract_geo_metadata.py -data_path ../../../data/GSE96997/GSE96997_series_matrix.txt -dataset_name cho2017 -skiprows 25

python extract_geo_metadata.py -data_path ../../../data/GSE49650/GSE49650_series_matrix.txt -dataset_name bristow2014 -skiprows 25



In [25]:
# Run bash file and print output message
shellscript = subprocess.run([f'./batch_metadata_file_{datetime_stamp}.sh'], stdin=subprocess.PIPE, stderr=subprocess.PIPE, stdout=subprocess.PIPE)
print(shellscript.stdout.decode('utf-8'))

saving to: ../../../data/GSE75694/GSE75694_metadata.tsv
	 condition GAL-SIC1Δ3P rep 1
	 condition GAL-SIC1Δ3P rep 2
	 condition GAL-SIC1Δ3P_nrm1Δyhp1Δyox1Δ rep 1
	 condition cdc16cdc28whi5stb1 rep 1
	 condition cdc16cdc28whi5stb1 rep 2
	 condition cdc28whi5stb1 rep 1
	 condition cdc28whi5stb1 rep 2
	 condition cdc28whi5stb1_elutr rep 1
	 condition cdc28whi5stb1_elutr rep 2
saving to: ../../../data/GSE8799/GSE8799_metadata.tsv
	 condition CyclinMutant rep 1
	 condition CyclinMutant rep 2
	 condition WildType rep 1
	 condition WildType rep 2
saving to: ../../../data/GSE32974/GSE32974_metadata.tsv
	 condition cdc28-4_37C rep 1
	 condition cdc28-4_37C rep 2
	 condition wild-type_37C rep 1
saving to: ../../../data/GSE96997/GSE96997_metadata.tsv
	 condition cdc14-3 rep 1
	 condition cdc15-2 rep 1
saving to: ../../../data/GSE49650/GSE49650_metadata.tsv
	 condition cdc20 rep 1
	 condition cdc20 rep 2
	 condition cdc8 rep 1
	 condition cdc8 rep 2
	 condition cdc8cdc20 rep 1
	 condition cse4 rep